In [7]:
import pandas as pd


df = pd.read_csv('ships.csv')
df.head()

,imo_number,mmsi,name,ship_type,length,width,max_draught,type_name,type_remark
0,9798351,215078000,ELDIA,80,149,34,8.0,Tanker,All ships of this type
1,9676125,257980000,EAGLE BARENTS,80,221,55,10.0,Tanker,All ships of this type
2,9429235,636020533,TUNADAL,74,148,10,7.7,Cargo,Hazardous category D
3,9372169,257282000,NORMAND SAPPHIRE,90,31,62,7.2,Other Type,All ships of this type
4,9639359,235094269,HIGHLAND CHIEFTAIN,79,15,65,4.8,Cargo,No additional information


# Auxiliary engine (AE) default power output (kW) assumptions (https://theicct.github.io/SAVE-doc/versions/v2025.03.1/#emissions-estimation-core-function)

In [8]:
cargo_specs_ax = {
    1: {
        'Capacity Range': [0, 4_999],
        'Berth': 90,
        'Anchor': 50,
        'Maneuver': 180,
        'Cruise': 60
    },
    2: {
        'Capacity Range': [5_000, 9_999],
        'Berth': 240,
        'Anchor': 130,
        'Maneuver': 490,
        'Cruise': 180
    },
    3: {
        'Capacity Range': [10_000, 19_999],
        'Berth': 720,
        'Anchor': 370,
        'Maneuver': 1450,
        'Cruise': 520
    },
    4: {
        'Capacity Range': [20_000, float('inf')],
        'Berth': 720,
        'Anchor': 370,
        'Maneuver': 1450,
        'Cruise': 520
    }
}

tanker_specs_ax = {
    1: {
        'Capacity Range': [0, 4999],
        'Berth': 250,
        'Anchor': 250,
        'Maneuver': 375,
        'Cruise': 250
    },
    2: {
        'Capacity Range': [5000, 9999],
        'Berth': 375,
        'Anchor': 375,
        'Maneuver': 560,
        'Cruise': 375
    },
    3: {
        'Capacity Range': [10000, 19999],
        'Berth': 690,
        'Anchor': 500,
        'Maneuver': 580,
        'Cruise': 490
    },
    4: {
        'Capacity Range': [20000, 59999],
        'Berth': 720,
        'Anchor': 520,
        'Maneuver': 600,
        'Cruise': 510
    },
    5: {
        'Capacity Range': [60000, 79999],
        'Berth': 620,
        'Anchor': 490,
        'Maneuver': 770,
        'Cruise': 560
    },
    6: {
        'Capacity Range': [80000, 119999],
        'Berth': 800,
        'Anchor': 640,
        'Maneuver': 910,
        'Cruise': 690
    },
    7: {
        'Capacity Range': [120000, 199999],
        'Berth': 2500,
        'Anchor': 770,
        'Maneuver': 1300,
        'Cruise': 860
    },
    8: {
        'Capacity Range': [200000, float('inf')],
        'Berth': 2500,
        'Anchor': 770,
        'Maneuver': 1300,
        'Cruise': 860
    }
}

# Boiler engine (BO) default power output (kW) assumptions (https://theicct.github.io/SAVE-doc/versions/v2025.03.1/#emissions-estimation-core-function)#

In [9]:
cargo_specs_bo = {
    1: {
        'Capacity Range': [0, 4_999],
        'Berth': 0,
        'Anchor': 0,
        'Maneuver': 0,
        'Cruise': 0
    },
    2: {
        'Capacity Range': [5_000, 9_999],
        'Berth': 110,
        'Anchor': 110,
        'Maneuver': 100,
        'Cruise': 0
    },
    3: {
        'Capacity Range': [10_000, 19_999],
        'Berth': 150,
        'Anchor': 150,
        'Maneuver': 130,
        'Cruise': 0
    },
    4: {
        'Capacity Range': [20_000, float('inf')],
        'Berth': 150,
        'Anchor': 150,
        'Maneuver': 130,
        'Cruise': 0
    }
}

tanker_specs_bo = {
    1: {
        'Capacity Range': [0, 4_999],
        'Berth': 500,
        'Anchor': 100,
        'Maneuver': 100,
        'Cruise': 0
    },
    2: {
        'Capacity Range': [5_000, 9_999],
        'Berth': 750,
        'Anchor': 150,
        'Maneuver': 150,
        'Cruise': 0
    },
    3: {
        'Capacity Range': [10_000, 19_999],
        'Berth': 1250,
        'Anchor': 250,
        'Maneuver': 250,
        'Cruise': 0
    },
    4: {
        'Capacity Range': [20_000, 59_999],
        'Berth': 2700,
        'Anchor': 270,
        'Maneuver': 270,
        'Cruise': 270
    },
    5: {
        'Capacity Range': [60_000, 79_999],
        'Berth': 3250,
        'Anchor': 360,
        'Maneuver': 360,
        'Cruise': 280
    },
    6: {
        'Capacity Range': [80_000, 119_999],
        'Berth': 4000,
        'Anchor': 400,
        'Maneuver': 400,
        'Cruise': 280
    },
    7: {
        'Capacity Range': [120_000, 199_999],
        'Berth': 6500,
        'Anchor': 500,
        'Maneuver': 500,
        'Cruise': 300
    },
    8: {
        'Capacity Range': [200_000, float('inf')],
        'Berth': 7000,
        'Anchor': 600,
        'Maneuver': 600,
        'Cruise': 300
    }
}

# Block coefficient
#### (https://yadda.icm.edu.pl/baztech/element/bwmeta1.element.baztech-82067de1-6e50-482c-86ae-235c5c012c9b/c/CHARCHALIS.pdf)

In [10]:
#since a vessel is not a perfect square we need to account for its shape

cargo_block_coef = 0.625
tanker_block_coef = 0.825

global cargo_block_coef
global tanker_block_coef


# Cargo

In [16]:
cargo = df[df['type_name'] == 'Cargo'][['type_name','length', 'width', 'max_draught']]
#displacement volume
cargo['dis_volume'] = cargo['length'] * cargo['width'] * cargo['max_draught'] * cargo_block_coef
cargo = cargo[cargo['dis_volume'] > 0]

#density of seawater 0.
cargo['dis_weight'] = cargo['dis_volume'] * 1.025

#DWT (Deadweight Tonnage) https://www.yumpu.com/en/document/read/11297041/empirical-ship-formula
#second variable that we subtract is the weight of the ship called the lightweight (empty ship)
cargo['DWT_cargo'] = cargo['dis_weight'] - (cargo['dis_weight']*0.32)


bins = [0]
for i in range(1,5):
    bins.append(cargo_specs_ax[i]['Capacity Range'][1])

cargo['DWT_cargo_AX'] = pd.cut(
    cargo['DWT_cargo'],
    bins=bins,
    labels=[f'{i}' for i in range(1, 5)],
    include_lowest=True,
    right=True
)

bins = [0]
for i in range(1,5):
    bins.append(cargo_specs_bo[i]['Capacity Range'][1])

cargo['DWT_cargo_BO'] = pd.cut(
    cargo['DWT_cargo'],
    bins=bins,
    labels=[f'{i}' for i in range(1, 5)],
    include_lowest=True,
    right=True
)


print(cargo[['DWT_cargo', 'DWT_cargo_AX', 'DWT_cargo_BO']].head())

        DWT_cargo DWT_cargo_AX DWT_cargo_BO
2     4964.382500            1            1
4     2038.725000            1            1
5    22425.975000            4            4
6   149367.753437            4            4
10  194706.601500            4            4


# Tanker

In [17]:
tanker = df[df['type_name'] == 'Tanker'][['type_name', 'length', 'width', 'max_draught']]
#displacemenet volume
tanker['dis_volume'] = tanker['length'] * tanker['width'] * tanker['max_draught'] * tanker_block_coef
tanker = tanker[tanker['dis_volume'] > 0]

#density of seawater https://os.copernicus.org/articles/5/91/2009/
tanker['dis_weight'] = tanker['dis_volume'] * 1.025

#DWT (Deadweight Tonnage) https://www.yumpu.com/en/document/read/11297041/empirical-ship-formula
#second variable that we subtract is the weight of the ship called the lightweight (empty ship)
tanker['DWT_tanker'] = tanker['dis_weight'] - (tanker['dis_weight']*0.16)

bins = [0]
for i in range(1,9):
    bins.append(tanker_specs_ax[i]['Capacity Range'][1])

tanker['DWT_tanker_AX'] = pd.cut(
    tanker['DWT_tanker'],
    bins=bins,
    labels=[f'{i}' for i in range(1, 9)],
    include_lowest=True,
    right=True
)

bins = [0]
for i in range(1,9):
    bins.append(tanker_specs_bo[i]['Capacity Range'][1])

tanker['DWT_tanker_BO'] = pd.cut(
    tanker['DWT_tanker'],
    bins=bins,
    labels=[f'{i}' for i in range(1, 9)],
    include_lowest=True,
    right=True
)

print(tanker[['DWT_tanker', 'DWT_tanker_AX', 'DWT_tanker_BO']].head())

      DWT_tanker DWT_tanker_AX DWT_tanker_BO
0   28788.051600             4             4
1   86340.003750             6             6
12  34905.512565             4             4
20   9349.652812             2             2
23    163.374750             1             1


### Data is based on this article (https://theicct.github.io/SAVE-doc/versions/v2025.03.1/)
### Block coefficient and calculation (https://yadda.icm.edu.pl/baztech/element/bwmeta1.element.baztech-82067de1-6e50-482c-86ae-235c5c012c9b/c/CHARCHALIS.pdf)